# This file makes an API call to get the data for all starlink statellites

## 1. dependencies

In [7]:
import requests
import json
import configparser
import xlsxwriter
import time
from datetime import datetime
import dsgp4

## 2. API building

In [8]:
uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query" 
requestFindStarlinks   = "/class/tle_latest/NORAD_CAT_ID/>40000/ORDINAL/1/OBJECT_NAME/STARLINK~~/format/json/orderby/NORAD_CAT_ID%20asc"

## 3. Read Config file

In [9]:
config = configparser.ConfigParser()
config.read("./SLTrack.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
configOut = config.get("configuration","output")
siteCred = {'identity': configUsr, 'password': configPwd}

## 4. Create an Excel File to store satellite data

In [10]:
now = datetime.now()
nowStr = now.strftime("%Y/%m/%d %H:%M:%S")

workbook = xlsxwriter.Workbook("TLE data")

worksheet = workbook.add_worksheet("starlink TLEs")

## 5. Edit the Spreadsheet with correct headers

In [11]:
worksheet.write('A1','NORAD_CAT_ID')#
worksheet.write('B1','OBJECT_NAME')#
worksheet.write('C1','EPOCH') #
worksheet.write('D1',"TLE_LINE0")#
worksheet.write('E1',"TLE_LINE1")#
worksheet.write('F1',"TLE_LINE2")#
wsline = 1

## 6. Making the API request and download the JSON file




In [12]:
class MyError(Exception):
    def __init___(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args



wsline = 1

with requests.Session() as session:
    # Log in
    resp = session.post(uriBase + requestLogin, data=siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")
    
    print("Login successful!")

    # Query for Starlink satellites
    print(uriBase + requestCmdAction + requestFindStarlinks) #print the satellite query for verification
    resp = session.get(uriBase + requestCmdAction + requestFindStarlinks)
    if resp.status_code != 200:
        print(resp)
        raise MyError(resp, "GET fail on request for Starlink satellites")

    print("Data retrieval successful!")

    # Parse response JSON
    retData = json.loads(resp.text)

    with open('Starlink_Satellites.json', 'w') as json_file: #save the json file
        json.dump(retData, json_file, indent=4)

    print("JSON data has been saved as 'Starlink_Satellites.json'")

Login successful!
https://www.space-track.org/basicspacedata/query/class/tle_latest/NORAD_CAT_ID/>40000/ORDINAL/1/OBJECT_NAME/STARLINK~~/format/json/orderby/NORAD_CAT_ID%20asc
Data retrieval successful!
JSON data has been saved as 'Starlink_Satellites.json'


In [13]:
import dsgp4
tles=dsgp4.tle.load('Starlink_Satellites.json')

ValueError: First line not compatible with TLE format.

## Convert the JSON into an excel file with only the TLE

In [ ]:
satCount = len(retData)
    print(f"Number of Starlink satellites retrieved: {satCount}")

    satIds = []

    # Loop through each satellite entry
    for e in retData:
        # Fetch satellite data safely with defaults
        catId = e.get('NORAD_CAT_ID', 'NA')
        Sat_name = e.get('OBJECT_NAME', 'NA')
        Epoch = e.get('EPOCH', 'NA')
        TLE_line_0 = e.get('TLE_LINE0', 'NA')
        TLE_line_1 = e.get('TLE_LINE1', 'NA')
        TLE_line_2 = e.get('TLE_LINE2', 'NA')

        print(f"Scanning satellite {Sat_name} at epoch {Epoch}")

        # Append NORAD ID to list
        satIds.append(catId)

        # Handle NORAD_CAT_ID being 'NA' to avoid ValueError on int()
        catId_value = int(catId) if catId != 'NA' else 'NA'

        # Write data to Excel worksheet
        worksheet.write(wsline, 0, catId_value)
        worksheet.write(wsline, 1, Sat_name)
        worksheet.write(wsline, 2, Epoch)
        worksheet.write(wsline, 3, TLE_line_0)
        worksheet.write(wsline, 4, TLE_line_1)
        worksheet.write(wsline, 5, TLE_line_2)

        wsline += 1  # Move to next row for the next satellite

# Close workbook after writing
workbook.close()

print("Completed session and closed workbook.")
